In [11]:
# %% [markdown]
# # Integración de Pinecone con modelo Logistic Regression para etiquetado multilabel

# %%
# Instalar librerías necesarias (una vez)
# %pip install pinecone
# %pip install sentence-transformers
# %pip install joblib

# %%
import os
import pandas as pd
import pinecone
import joblib
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from pinecone import Pinecone, ServerlessSpec
import configparser
import time
from pinecone import exceptions


DEFAULT_BOOK_TITLE = "Messi: Edición revisada y actualizada (Biografías y memorias)"
DEFAULT_BOOK_BLURB = (
    "Leo Messi es el jugador de fútbol más conocido del planeta, pero también un enigma como persona, por su hermetismo. Esta biografía, que fue publicada por primera vez en 2014, y posteriormente actualizada en 2018, se presenta de nuevo en una edición que recoge lo más relevante de los últimos años del jugador en el Fútbol Club Barcelona. \n\n"
    "En esta nueva edición, el autor repasa lo más destacado desde aquel fatídico Mundial de Brasil hasta el final de la temporada 2017/18, así como su paso por el Mundial de Rusia y por la Copa América 2021, que coincidía con el momento en que expiraba su contrato con el Fútbol Club Barcelona, y que convirtió al astro argentino en foco de todas las miradas, generando una enorme expectación.\n\n"
    "En agosto de 2021, se anunció el desenlace que parecía imposible: Messi no pudo renovar en el Barça y se anunció su fichaje por el PSG. ¿Qué pasó? ¿Cómo es posible que, queriendo quedarse, tuviera que salir?"
)

In [12]:
# Get the pinecone API key from config.cfg
config = configparser.ConfigParser()
config.read('../config.cfg')
PINECONE_API_KEY = config['pinecone']['api_key']
PINECONE_ENV = config['pinecone']['environment']

# %%
# 1. Inicializa Pinecone (usa tu API Key personal desde https://app.pinecone.io/)
try:
    pc = Pinecone(api_key=PINECONE_API_KEY)
    print(f"✅ Connected to Pinecone!")
except Exception as e:
    print("❌ Failed to connect:", e)

index_name = "book-embeddings"
#index_name = "quickstart"

from pinecone import exceptions

try:
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(cloud='aws', region='us-east-1')
    )
    print("✅ Índice creado correctamente.")
except exceptions.PineconeApiException as e:
    if "ALREADY_EXISTS" in str(e):
        print("📌 El índice ya existe. Usando el existente.")
    else:
        raise e


index = pc.Index(index_name)

# Borrar vectores del índice si ya había
stats = index.describe_index_stats()
try:
    index.delete(delete_all=True, namespace="books")
    print("🗑️ Vectores antiguos en 'books' borrados.")
except pinecone.exceptions.NotFoundException:
    print("ℹ️ Namespace 'books' no encontrado. Nada que borrar.")


✅ Connected to Pinecone!
📌 El índice ya existe. Usando el existente.
🗑️ Vectores antiguos en 'books' borrados.


In [13]:
# 3. Carga tus modelos ya entrenados
model = joblib.load("../model/book_tagging_pipeline_sentence_bert.joblib")
clf = joblib.load("../model/book_tagging_pipeline.joblib")
mlb = joblib.load("../model/book_tagging_pipeline_mlb.joblib")

# 4. Carga los libros de Goodreads y sube a Pinecone
df = pd.read_csv("../data/raw/goodreads_data.csv")  # o reemplaza con la ruta real si cambió

# Normaliza columnas necesarias
df['Book'] = df['Book'].fillna('')
df['Description'] = df['Description'].fillna('')
df['Genres'] = df['Genres'].fillna("[]")

# Une título y descripción
df['text'] = df['Book'] + ". " + df['Description']

# Limpia los géneros
def parse_genres(raw_genres):
    try:
        genres = eval(raw_genres)
        if isinstance(genres, list):
            return ','.join([g.strip() for g in genres])
    except:
        pass
    return ''

df['tags'] = df['Genres'].apply(parse_genres)

# Muestra muestra para confirmar
print(df[['Book', 'tags']].head(3))

                                                Book  \
0                              To Kill a Mockingbird   
1  Harry Potter and the Philosopher’s Stone (Harr...   
2                                Pride and Prejudice   

                                                tags  
0  Classics,Fiction,Historical Fiction,School,Lit...  
1  Fantasy,Fiction,Young Adult,Magic,Childrens,Mi...  
2  Classics,Fiction,Romance,Historical Fiction,Li...  


In [14]:
# Guardar embeddings para evitar recomputarlos en el futuro
if os.path.exists("../model/vectors.npy"):
    X_embeddings = np.load("../model/vectors.npy")
    print("📥 Embeddings cargados desde ../model/vectors.npy")
else:
    X_embeddings = model.encode(df['text'].tolist(), show_progress_bar=True)
    np.save("../model/vectors.npy", X_embeddings)
    print("💾 Embeddings guardados en ../model/vectors.npy")

📥 Embeddings cargados desde ../model/vectors.npy


In [15]:
# Prepare data for Pinecone
pinecone_data = []
for idx, (vec, tags) in enumerate(zip(X_embeddings, df['tags'])):
    # Ensure tags are strings and handle NaN or invalid values
    if isinstance(tags, list):
        tags = ','.join(tags)  # Convert list to comma-separated string
    elif pd.isna(tags) or not isinstance(tags, str):
        tags = ''  # Replace NaN or invalid values with an empty string

    pinecone_data.append((
        str(idx),
        vec.tolist(),
        {"tags": tags}
    ))

# Upload embeddings to Pinecone in batches of 1000
batch_size = 1000
for i in range(0, len(pinecone_data), batch_size):
    batch = pinecone_data[i:i + batch_size]
    # index.upsert(vectors=batch)
    index.upsert(vectors=batch, namespace="books")


print("✅ Embeddings uploaded to Pinecone.")

# Ver cuántos vectores hay almacenados
# stats = index.describe_index_stats()
stats = index.describe_index_stats(namespace="books")
print(f"📊 Vectores cargados en el índice: {stats['total_vector_count']}")

# Comprueba que la consulta devuelve algo:
query_vector = X_embeddings[0]  # el primer libro
# res = index.query(vector=query_vector.tolist(), top_k=3, include_metadata=True)
res = index.query(vector=query_vector.tolist(), top_k=3, include_metadata=True, namespace="books")
print(f"res = {res}")

✅ Embeddings uploaded to Pinecone.
📊 Vectores cargados en el índice: 10000
res = {'matches': [{'id': '0',
              'metadata': {'tags': 'Classics,Fiction,Historical '
                                   'Fiction,School,Literature,Young '
                                   'Adult,Historical'},
              'score': 1.0,
              'values': []},
             {'id': '439',
              'metadata': {'tags': 'Classics,Fiction,Historical '
                                   'Fiction,Literature,High '
                                   'School,School,Literary Criticism'},
              'score': 0.811565,
              'values': []},
             {'id': '5638',
              'metadata': {'tags': 'Science Fiction,Fiction,Dystopia,Post '
                                   'Apocalyptic,Speculative '
                                   'Fiction,Robots,Science Fiction Fantasy'},
              'score': 0.699109674,
              'values': []}],
 'namespace': 'books',
 'usage': {'read_units'

In [16]:
# 5. Función de predicción combinada (ensemble)

def predict_with_ensemble(title, blurb, top_k=5, pinecone_max_tags=6, threshold=0.3):
    text = title + ". " + blurb
    embedding = model.encode([text])[0]
    
    # A. Predict con Logistic Regression
    probs = np.array([estimator.predict_proba(embedding.reshape(1, -1))[0][1]
                      for estimator in clf.estimators_])
    pred_lr = (probs >= threshold).astype(int)
    pred_lr = np.array([pred_lr])
    tags_lr = sorted(mlb.inverse_transform(pred_lr)[0])
    
    # B. Predict con Pinecone (namespace fijo y tags en minúscula)
    pinecone_result = index.query(
        vector=embedding.tolist(),
        top_k=top_k,
        include_metadata=True,
        namespace="books"
    )
    
    seen = set()
    tags_pinecone = []
    for match in pinecone_result.matches:
        if 'tags' in match.metadata and match.metadata['tags']:
            for tag in match.metadata['tags'].split(','):
                tag = tag.strip().lower()
                if tag not in seen:
                    seen.add(tag)
                    tags_pinecone.append(tag)
                    if len(tags_pinecone) >= pinecone_max_tags:
                        break
        if len(tags_pinecone) >= pinecone_max_tags:
            break

    # C. Fusión inteligente (con prioridad a los de Logistic)
    final_tags = list(tags_lr)
    for tag in tags_pinecone:
        if tag not in final_tags:
            final_tags.append(tag)

    return {
        "tags_logistic": tags_lr,
        "tags_pinecone": tags_pinecone,
        "tags_fusion": final_tags
    }

# 6. Prueba con libro desconocido para ti
result = predict_with_ensemble(DEFAULT_BOOK_TITLE, DEFAULT_BOOK_BLURB)

print("Tags por Logistic Regression:", result["tags_logistic"])
print("Tags por Pinecone:", result["tags_pinecone"])
print("Tags combinados (fusión):", result["tags_fusion"])

Tags por Logistic Regression: ['biography']
Tags por Pinecone: ['fiction', 'thriller', 'mystery', 'espionage', 'japan', 'mystery thriller']
Tags combinados (fusión): ['biography', 'fiction', 'thriller', 'mystery', 'espionage', 'japan', 'mystery thriller']
